In [7]:
#Author : Keval Kavle
#Date : 12-03-2020
#Subject : Cloud Computing CS643
#Assignment : Programming Assignment-2

In [8]:
# Importing Libraries
from pyspark.sql import SparkSession
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.sql.functions import ceil, col
import pyspark.sql.functions as func
from pyspark.ml.regression import LinearRegression
from pyspark.ml.classification import RandomForestClassifier
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
import sys

In [9]:
#SparkConf().getAll()
##### Opening a Spark Session  #####
spark = SparkSession.builder.master("local").appName("WineQualityPrediction-CS643").getOrCreate()

In [10]:
#### Training Dataset
trainDf = spark.read.csv('TrainingDataset.csv',header='true', inferSchema='true', sep=';')

In [11]:
from pyspark.sql.functions import isnan, when, count, col

#trainDf.select([count(when(isnan(c), c)).alias(c) for c in trainDf.columns]).show()

In [12]:
#trainDf

In [13]:
#trainDf.show(5,truncate=False)
trainDf = trainDf.withColumnRenamed(trainDf.columns[-1],'quality')

In [14]:
#trainDf.printSchema()

In [15]:
#trainDf.show(2)

In [16]:
from pyspark.sql.types import *

schemaTrain = trainDf.schema

In [17]:
##### Test Dataset ####
if (sys.argv[1] == '' or sys.argv[1] == "-f"):
    testpath = "./"
else :
    testpath = sys.argv[1]

print(testpath)
testpath = "C:/Users/DELL/Python_Practice/CS643-ProgrammingAssignment2/"
testfile = testpath + 'ValidationDataset.csv'
valDf = spark.read.csv(testfile,header='true', schema=schemaTrain, sep=';') 

./


In [18]:
#valDf.show(2)

In [19]:
valDf = valDf.withColumnRenamed(valDf.columns[-1],'quality')

In [20]:
#valDf.show(5)

In [21]:
trainColumns = [c for c in trainDf.columns  if "quality" not in c  ]
#trainColumns

In [22]:
trainAssembler = VectorAssembler(inputCols=trainColumns,outputCol="independentFeatures")

In [23]:
trainTrans = trainAssembler.transform(trainDf)

In [24]:
#train = trainTrans

In [25]:
#trainTrans.show(2)

In [26]:
#trainTrans.select("independentFeatures").show(2,truncate=False)

In [27]:
standardScaler = StandardScaler(inputCol="independentFeatures",outputCol="scaledFeatures")

In [28]:
train = standardScaler.fit(trainTrans).transform(trainTrans)

In [29]:
#train.show(2,truncate=False)
#train.select("scaledFeatures").show(2,truncate=False)

In [30]:
#train.show(2,truncate=False)

In [31]:
valColumns = [c for c in valDf.columns  if "quality" not in c  ]
#valColumns

In [32]:
valAssembler = VectorAssembler(inputCols=valColumns,outputCol="independentFeatures")

In [33]:
valTrans = valAssembler.transform(valDf)

In [34]:
#valTrans.select("independentFeatures").show(2,truncate=False)

In [35]:
standardScaler = StandardScaler(inputCol="independentFeatures",outputCol="scaledFeatures")

In [36]:
val = standardScaler.fit(valTrans).transform(valTrans)

In [37]:
#val = valTrans

In [38]:
##### Random Forest  #####
rf = RandomForestClassifier(labelCol="quality", featuresCol="independentFeatures",numTrees=10)

In [39]:
model = rf.fit(train)

In [40]:
model.write().overwrite().save("rfModel")

In [41]:
predictions = model.transform(val)
#predictions.show(2)

In [42]:
predictions = predictions.withColumn("prediction", func.round("prediction"))

In [43]:
##### Random Forest  Ends #####

In [44]:
#### Linear Regression  #####
regressor = LinearRegression(featuresCol="independentFeatures",labelCol="quality")
regressor=regressor.fit(train)

In [45]:
predResults = regressor.evaluate(val)

In [46]:
predResults = predResults.predictions

In [47]:
regressor.write().overwrite().save("lrModel")

In [48]:
predResults = predResults.withColumn("prediction", func.round("prediction"))
#predResults.show(2)

In [49]:
##### Linear Regression Ends ######

In [50]:
#predResults.show()

In [51]:
#val.show(2)

In [52]:
#predictions.show(2)

In [53]:
#roundoffPrediction =  predictions.select("prediction", func.round(col('prediction')))
#roundoffPrediction.show(2)

In [54]:
#predictions = predictions.withColumn("predictionFinal", roundoffPrediction[1])
#predictions = predictions.withColumn("prediction", func.round("prediction"))

In [55]:
#predictions.show()

In [56]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [57]:
####### Random Forest Accuarcy - not using this as simple linear regression is giving better results ############
evalAcc = MulticlassClassificationEvaluator(
    labelCol="quality", predictionCol="prediction", metricName="accuracy")
accuracy = evalAcc.evaluate(predictions)

##print("accuracy Test Error = %g" % (1.0 - accuracy))


transformed_data = model.transform(val)
transformed_data = transformed_data.withColumn("prediction", func.round("prediction"))
##print(evalAcc.getMetricName(), 'accuracy:', evalAcc.evaluate(transformed_data))

In [58]:
####### Random Forest f1 - not using this as simple linear regression is giving better results ############
evalVal = MulticlassClassificationEvaluator(
    labelCol="quality", predictionCol="prediction", metricName="f1")
accuracy = evalVal.evaluate(predictions)
##print("f1 Test Error = %g" % (1.0 - accuracy))
transformed_data = model.transform(val)
transformed_data = transformed_data.withColumn("prediction", func.round("prediction"))
##print(evalVal.getMetricName(), 'accuracy :', evalVal.evaluate(transformed_data))

In [59]:
####### Linear Regression Accuarcy and f1 ############


# Create evaluator
evaluatorMulti = MulticlassClassificationEvaluator(labelCol="quality", predictionCol="prediction")

# Make predicitons
predictionAndTarget = regressor.transform(val).select("quality", "prediction")
predictionAndTarget = predictionAndTarget.withColumn("prediction", func.round("prediction"))
# Get metrics
acc = evaluatorMulti.evaluate(predictionAndTarget, {evaluatorMulti.metricName: "accuracy"})
f1 = evaluatorMulti.evaluate(predictionAndTarget, {evaluatorMulti.metricName: "f1"})

In [60]:
print("Accuracy :" , acc * 100 , "%")
print("f1 Score :" , f1)

Accuracy : 60.0 %
f1 Score : 0.5846181534031478
